# OI Back-Analysis Demo

This notebook showcases how to explore historical option-chain data that
has been captured by the web application.

**Workflow**
- Configure the exchange and date range you want to analyse.
- Fetch raw snapshots from `oi_tracker.db` using `OIBackAnalysisPipeline`.
- Build summary tables that align call/put OI with the underlying index.
- Visualise the results with prebuilt Plotly helpers.

> ℹ️ Ensure the database has recent data before running the notebook. You
> can export fresh samples by running the web app for a trading session.


In [1]:
import sys
from pathlib import Path

# Ensure the project root is on the Python path so that
# `analysis` (and other in-repo packages) can be imported.
NOTEBOOK_DIR = Path().resolve()
if NOTEBOOK_DIR.name == "notebooks":
    PROJECT_ROOT = NOTEBOOK_DIR.parent
else:
    PROJECT_ROOT = NOTEBOOK_DIR

if PROJECT_ROOT.name != "BSE_OI_Analysis":
    PROJECT_ROOT = PROJECT_ROOT / ".."
    PROJECT_ROOT = Path(PROJECT_ROOT).resolve()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Python path initialised with project root: {PROJECT_ROOT}")


Python path initialised with project root: /Users/kpal/projects/BSE_OI_Analysis


In [28]:
from __future__ import annotations

from datetime import datetime, timedelta

EXCHANGE = "NSE"  # or "BSE"
# Analyse the last trading day by default
END_TIME = "2025-11-05 15:30"  # datetime.now()
START_TIME = "2025-11-05 09:15"  # END_TIME - timedelta(days=1)

DB_PATH = PROJECT_ROOT / "oi_tracker.db"
print(f"Exchange: {EXCHANGE}")
if isinstance(START_TIME, datetime) and isinstance(END_TIME, datetime):
    start_str = START_TIME.strftime("%Y-%m-%d %H:%M")
    end_str = END_TIME.strftime("%Y-%m-%d %H:%M")
else:
    start_str = str(START_TIME)
    end_str = str(END_TIME)
print(f"Time window: {start_str} → {end_str}")
print(f"Database path: {DB_PATH}")


Exchange: NSE
Time window: 2025-11-05 09:15 → 2025-11-05 15:30
Database path: /Users/kpal/projects/BSE_OI_Analysis/oi_tracker.db


In [26]:
import pandas as pd

from analysis import OIBackAnalysisPipeline
from analysis.visualizations import (
    plot_heatmap,
    plot_net_oi_vs_underlying_change,
    plot_underlying_vs_oi,
)

pipeline = OIBackAnalysisPipeline(db_path=DB_PATH)
print(f"Using database: {pipeline.db_path}")

snapshots = pipeline.fetch_snapshots(
    exchange=EXCHANGE,
    start=START_TIME,
    end=END_TIME,
)

print(f"Fetched {len(snapshots):,} snapshot rows")
snapshots.head()

output_path = PROJECT_ROOT / "exports" / "snapshots_2025-11-09_NSE.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
snapshots.to_csv(output_path, index=False)
print(f"Saved {len(snapshots):,} rows to {output_path}")

Using database: /Users/kpal/projects/BSE_OI_Analysis/oi_tracker.db
Fetched 0 snapshot rows
Saved 0 rows to /Users/kpal/projects/BSE_OI_Analysis/exports/snapshots_2025-11-09_NSE.csv


In [30]:
try:
    snapshots
except NameError:
    print("Run the data-fetch cell first to populate `snapshots`.")
else:
    if snapshots.empty:
        print("No snapshots found.")
        summary = pd.DataFrame()
    else:
        summary = pipeline.build_summary_table(
            snapshots,
            resample_rule="5min",
            include_pct_changes=True,
        )
        
        print(f"Summary rows: {len(summary):,}")
    display(summary.head())

No snapshots found.


""


In [17]:
if summary.empty:
    print("Skip lagged features: summary is empty.")
    lagged = pd.DataFrame()
else:
    lagged = pipeline.build_lagged_features(
        summary,
        columns=["net_oi", "underlying_return_pct"],
        lags=[1, 2, 3],
    )

lagged.head()


,timestamp,ce_oi,pe_oi,underlying_price,underlying_change,underlying_return_pct,net_oi,total_oi,pcr,ce_oi_atm,...,pe_pct_change_5m_weighted,pe_pct_change_10m_weighted,pe_pct_change_15m_weighted,pe_pct_change_30m_weighted,net_oi_lag_1,net_oi_lag_2,net_oi_lag_3,underlying_return_pct_lag_1,underlying_return_pct_lag_2,underlying_return_pct_lag_3
0,2025-11-08 09:15:00,53857350.0,59779725.0,25492.3,0.0,0.0,0.0,0.0,NaN,8441475.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-08 09:20:00,53857350.0,59779725.0,25492.3,0.0,0.0,0.0,0.0,NaN,8441475.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN
2,2025-11-08 09:25:00,53857350.0,59779725.0,25492.3,0.0,0.0,0.0,0.0,NaN,8441475.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN
3,2025-11-08 09:30:00,53857350.0,59779725.0,25492.3,0.0,0.0,0.0,0.0,NaN,8441475.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2025-11-08 09:35:00,53857350.0,59779725.0,25492.3,0.0,0.0,0.0,0.0,NaN,8441475.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
if summary.empty:
    print("No summary data available, skipping line charts.")
else:
    fig1 = plot_underlying_vs_oi(summary, exchange=EXCHANGE)
    fig1.show()

    fig2 = plot_net_oi_vs_underlying_change(summary, exchange=EXCHANGE)
    fig2.show()

if snapshots.empty:
    print("No snapshots available, skipping heatmap.")
else:
    heatmap_matrix = pipeline.build_strike_heatmap(
        snapshots,
        option_type="CE",
        value_column="pct_change_5m",
    )
    if heatmap_matrix.empty:
        print("Heatmap matrix is empty – try a different option type or timeframe.")
    else:
        fig3 = plot_heatmap(
            heatmap_matrix,
            title_suffix=f"{EXCHANGE} Calls – 5m Δ OI",
        )
        fig3.show()
